In [3]:
import pandas as pd


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [16]:
Data = spark.read.format("csv").option("header", "true").load("Youtube.csv")
 
Data.createOrReplaceTempView("datastats")
 
Data.show()

+----+--------------------+-----------+------------+----------------+--------------------+-------+--------------+------------+-------------+----------------+------------+-----------------+--------------------------------+-----------------------+------------------------+----------------------+-----------------------+----------------------------+------------+-------------+------------+---------------------------------------+----------+-----------------+----------------+---------+-----------+
|rank|            Youtuber|subscribers| video views|        category|               Title|uploads|       Country|Abbreviation| channel_type|video_views_rank|country_rank|channel_type_rank|video_views_for_the_last_30_days|lowest_monthly_earnings|highest_monthly_earnings|lowest_yearly_earnings|highest_yearly_earnings|subscribers_for_last_30_days|created_year|created_month|created_date|Gross tertiary education enrollment (%)|Population|Unemployment rate|Urban_population| Latitude|  Longitude|
+----+----

In [17]:
spark.sql('''
    SELECT 
        Youtuber,
        AVG(highest_monthly_earnings - lowest_monthly_earnings) AS average_earnings_variation
    FROM 
        datastats 
    WHERE 
        highest_monthly_earnings IS NOT NULL 
        AND lowest_monthly_earnings IS NOT NULL 
        AND (highest_monthly_earnings + lowest_monthly_earnings) != 0
    GROUP BY 
        Youtuber 
    ORDER BY 
        average_earnings_variation DESC
''').show()


+--------------------+--------------------------+
|            Youtuber|average_earnings_variation|
+--------------------+--------------------------+
|            T-Series|                 8435400.0|
| KL BRO Biju Rithvik|                 7591900.0|
|Cocomelon - Nurse...|                 7406200.0|
|           SET India|                 6844100.0|
|              Zee TV|                 6373200.0|
|            StarPlus|                 6283200.0|
|            Sony SAB|                 6185700.0|
|         GR6 EXPLODE|                 6091300.0|
|ZAMZAM ELECTRONIC...|                 6000400.0|
|           BETER B  |                 5534100.0|
|            ViralHog|                 5159100.0|
|         BeatboxJCOP|                 5156200.0|
|             MrBeast|                 5063000.0|
|         HAR PAL GEO|                 4965700.0|
|           dednahype|                 4688800.0|
|           PANDA BOI|                 4593600.0|
|           Colors TV|                 4503100.0|


In [23]:
#2)	Evaluate how efficiently each video generates revenue based on views.
spark.sql('''
    SELECT
        Youtuber,
        Title,
        `video views` AS video_views,
        (highest_monthly_earnings - lowest_monthly_earnings) / NULLIF(`video views`, 0) AS revenue_per_view
    FROM
        datastats
    WHERE
        highest_monthly_earnings IS NOT NULL
        AND lowest_monthly_earnings IS NOT NULL
        AND `video views` IS NOT NULL
    ORDER BY
        revenue_per_view DESC
''').show()


+--------------------+--------------------+-----------+--------------------+
|            Youtuber|               Title|video_views|    revenue_per_view|
+--------------------+--------------------+-----------+--------------------+
|            Vibhu 96|            Vibhu 96|   20563378|0.003695890821050...|
|Super JoJo - Nurs...|Super JoJo - Nurs...|  329774870|0.003693125555625...|
|          Go Ami Go!|          Go Ami Go!| 1714955279|7.102809122289655E-4|
|ZAMZAM ELECTRONIC...|ZAMZAM ELECTRONIC...| 8507818877|7.052806467497157E-4|
|         BeatboxJCOP|         BeatboxJCOP| 8716982055|5.915120585848218E-4|
|      Manoj  parihar|      Manoj  parihar|  264228052|5.275745665339122E-4|
|        DEV Ke Facts|        DEV Ke Facts| 1971226335|5.233290473465595E-4|
|       Jess No Limit|       Jess No Limit| 3963007415| 5.20614721080455E-4|
|                GH'S|                GH'S| 8739174649|4.611076173493234...|
|            _vector_|            _vector_| 7597013023|4.441350817465882E-4|

In [27]:
#ChannelswithHighEngagementRelativetoSubscribers
spark.sql(" SELECT Youtuber, `video views` / subscribers * 100 AS engagement_per_subscriber FROM datastats ORDER BY engagement_per_subscriber DESC LIMIT 10").show()

+----------------+-------------------------+
|        Youtuber|engagement_per_subscriber|
+----------------+-------------------------+
|        netd m  |        232805.3423568465|
|     jbalvinVEVO|        157148.3833409091|
|    Ryan's World|       157101.81874431818|
|Vijay Television|       144148.07166666666|
|     Maroon5VEVO|        138224.6310275862|
|     shakiraVEVO|       137864.81680434782|
|    Tsuriki Show|       136932.66550869567|
|        ViralHog|       135126.33673571429|
|        LankyBox|       135041.26797637795|
|      MalumaVEVO|        134850.6594397163|
+----------------+-------------------------+



In [28]:
#Analyzetheperformanceofvariouschannelcategoriesbyassessingthetotalnumberofvideoviewstheygeneratedinthe last 30days.
spark.sql(" SELECT channel_type, SUM(video_views_for_the_last_30_days) AS total_views_last_30_days FROM datastats GROUP BY channel_type").show()


+-------------+------------------------+
| channel_type|total_views_last_30_days|
+-------------+------------------------+
|        Howto|           2.127236291E9|
|    Education|              9.220914E9|
|Entertainment|         6.1557841055E10|
|        Autos|             3.5326153E8|
|       Sports|           2.128495004E9|
|         Film|           5.763538061E9|
|        Games|           8.598094041E9|
|         Tech|            9.36521005E8|
|    Nonprofit|               1.95181E8|
|        Music|         3.7532535662E10|
|      Animals|              2.122043E9|
|             |                162732.0|
|       People|         1.3803361493E10|
|       Comedy|           9.600746623E9|
|         News|           5.250125924E9|
+-------------+------------------------+



In [29]:
# AverageViewsperVideoforTop3EducationChannels
spark.sql("SELECT Youtuber, `video views` / uploads AS avg_views_per_video FROM datastats WHERE category = 'Education'ORDER BY avg_views_per_video DESC LIMIT 3").show()


+--------------------+--------------------+
|            Youtuber| avg_views_per_video|
+--------------------+--------------------+
|Khan GS Research ...|        2.07307307E9|
|Cocomelon - Nurse...|1.6977225672877848E8|
|         Jason Vlogs|1.0727286120481928E8|
+--------------------+--------------------+



In [30]:
# Top5ChannelswiththeMostMonthlyEarningsperVideo
spark.sql("SELECT Youtuber, highest_monthly_earnings / uploads AS earnings_per_video FROM datastats ORDER BY earnings_per_video DESC LIMIT 5").show()


+--------------------+------------------+
|            Youtuber|earnings_per_video|
+--------------------+------------------+
|     Michael Jackson|42293.333333333336|
|               Adele|           36560.0|
|Super JoJo - Nurs...| 36111.11111111111|
|  Galinha Pintadinha|24242.424242424244|
|             KAROL G| 14179.10447761194|
+--------------------+------------------+



In [31]:
# Identify channels that have experienced substantial growth over the past year by analyzing the percentage change insubscribers
spark.sql("SELECT Youtuber, ((subscribers - subscribers_for_last_30_days) / subscribers_for_last_30_days) * 100 AS yearly_growth_rate FROM datastats ORDER BY yearly_growth_rate DESC LIMIT 5").show()


+------------------+------------------+
|          Youtuber|yearly_growth_rate|
+------------------+------------------+
|       BRIGHT SIDE|       4.4499999E9|
|    MrBeast Gaming|       3.6099999E9|
|             Drake|       2.8099999E9|
|Sandeep Maheshwari|       2.7799999E9|
|          FaZe Rug|       2.3699999E9|
+------------------+------------------+



In [32]:
#ChannelswiththeHighestSubscribers-to-ViewsRatio
spark.sql("SELECT Youtuber, subscribers / `video views` AS subscribers_to_views_ratio FROM datastats ORDER BY subscribers_to_views_ratio DESC LIMIT 3 ").show()


+-------------+--------------------------+
|     Youtuber|subscribers_to_views_ratio|
+-------------+--------------------------+
|  Happy Lives|         8807.896735003796|
|       YYYYYY|         33.93319942245239|
|Calon Sarjana|         1.218987893106014|
+-------------+--------------------------+

